In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers, activations
from tensorflow.keras.layers import Conv2D, TimeDistributed, BatchNormalization, MaxPooling2D, Flatten, Bidirectional, Dense,Dropout,Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
x_data = np.load('data/X_train.npy') 
y_data = np.load('data/y_train.npy') 

In [3]:
print(x_data.shape, y_data.shape)

(2520, 5, 100, 200, 3) (2520, 11)


In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(x_data, y_data, test_size = 0.2, shuffle = True, stratify=y_data)
print("x_train: {}, X_test: {}".format(X_train.shape, X_test.shape))
print("Y_train: {}, Y_test: {}".format(Y_train.shape, Y_test.shape))

x_train: (2016, 5, 100, 200, 3), X_test: (504, 5, 100, 200, 3)
Y_train: (2016, 11), Y_test: (504, 11)


In [5]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Input


# this could also be the output a different Keras model or layer
#input_tensor = Input(shape=(224, 224, 3))

#model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)

class MobileNet_LSTM(keras.Model):
    def __init__(self, num_classes):
        super(MobileNet_LSTM, self).__init__()
        #self.BZ = input_shape[0]
        #self.frame = input_shape[1]
        
        self.mobile = TimeDistributed(MobileNet(weights='imagenet', include_top=False))
        #self.vgg.trainable = False

        self.bilstm = layers.Bidirectional(layers.LSTM(256), merge_mode='concat')
        self.dense = layers.Dense(num_classes, activation='softmax')
        self.max = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))
        self.dropout = layers.Dropout(0.2)
        self.flat= TimeDistributed(Flatten())

    def call(self, x):
        x = self.mobile(x)
        x = self.max(x)
        x = self.dropout(x)      
        x = self.flat(x)
        x = self.dropout(x)
        x = self.bilstm(x)
        return self.dense(x)

In [8]:
model = MobileNet_LSTM(num_classes = 11)
adam = tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

C:\Users\hwj43\anaconda3\envs\Lip\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [7]:
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [9]:
from keras.callbacks import ModelCheckpoint
checkpoint  = ModelCheckpoint('ModelCheckpoint/',monitor='val_loss', save_weights_only=True, mode='auto')
early_stopping_patience = 10
# Add early stopping
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=early_stopping_patience, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, Y_train, batch_size=16, epochs=30, verbose = 1, validation_data = (X_test, Y_test),callbacks=[checkpoint, early_stopping])

C:\Users\hwj43\anaconda3\envs\Lip\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/30
126/126 [==============================] - 3387s 27s/step - loss: 1.2647 - accuracy: 0.5878 - val_loss: 0.7445 - val_accuracy: 0.7083
Epoch 2/30
126/126 [==============================] - 2460s 19s/step - loss: 0.5969 - accuracy: 0.7882 - val_loss: 0.5419 - val_accuracy: 0.7817
Epoch 3/30
126/126 [==============================] - 2248s 18s/step - loss: 0.3987 - accuracy: 0.8690 - val_loss: 0.4917 - val_accuracy: 0.7857
Epoch 4/30
126/126 [==============================] - 2355s 19s/step - loss: 0.2819 - accuracy: 0.9127 - val_loss: 0.4247 - val_accuracy: 0.8532
Epoch 5/30
114/126 [==========================>...] - ETA: 4:42 - loss: 0.1938 - accuracy: 0.9534